# Scraping

This notebook follows an example of scraping Nadlan Gov. It yeilds a query csv in 'data/gov' folder which will be avilable of off line analysis. 

Follow the comments to modify and try different queries.

**Note:** This proceedure takes time, queries with a mutiple results could take a long time. Try not to query about more than one neigberhood.

In [ ]:
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains

# import org.openqa.selenium.JavascriptExecutor as JavascriptExecutor
import pprint
import pandas as pd

In [ ]:
# insert your q
q = 'רמת בגין חיפה'


In [ ]:
# update to your driver path
DRIVER_PATH = '/chromedriver'

q = '%20'.join(q.split(' '))
url = f'https://www.nadlan.gov.il/?search={q}'
browser = webdriver.Chrome(executable_path=DRIVER_PATH)
browser.get(url)

# now wait for load
browser.implicitly_wait(5)

browser.set_window_size(1500, 1000)

In [ ]:
class infinite_scroll(object):
   
    def __init__(self, last):
    
        self.last = last

    def __call__(self, driver):
        new = driver.execute_script('return document.body.scrollHeight')  
        if new > self.last:
            return new
        else:
            return False
    

In [ ]:
last_height = browser.execute_script('return document.body.scrollHeight')
print(last_height)
flag=1

while flag==1:
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    sleep(randint(8,12))

    try:
        w = wait(browser, 10)

        new_height = w.until(infinite_scroll( last_height))
        last_height = new_height

    except:
        print("End of page reached")
        flag = 0

In [ ]:
table = browser.find_elements_by_class_name("tableRow")
len(table)

In [ ]:
def expand_and_read(r):
    
    r.find_elements_by_tag_name("button")[0].click()
    y = r.find_elements_by_class_name("value")
    col_dict = dict()
    col_dict['שנת בנייה מבנה'] = y[0].text
    col_dict['שנת בנייה דירה'] = y[1].text
    col_dict['קומות במבנה'] = y[2].text
    
    return col_dict

In [ ]:
relevant_col_names = ['קומות במבנה','שנת בנייה דירה','שנת בנייה מבנה','תאריך מכירה', 'כתובת', 'גוש חלקה -תת חלקה', 'סוג נכס', 'חדרים', 'קומה', 'מ"ר','סכום']
relevant_col_dict = {c:[] for c in relevant_col_names}

In [ ]:
for idx in range(len(table)):
    print(idx)
    row = table[idx]
    cols = row.find_elements_by_class_name("tableCol")

    # naive map - in absence of unique tags or title per col
    relevant_col_dict['תאריך מכירה'].append(cols[0].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['כתובת'].append(cols[1].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['גוש חלקה -תת חלקה'].append(cols[2].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['סוג נכס'].append(cols[3].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['חדרים'].append(cols[4].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['קומה'].append(cols[5].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['מ"ר'].append(cols[6].find_elements_by_tag_name("div")[0].text)
    relevant_col_dict['סכום'].append(cols[7].find_elements_by_tag_name("div")[0].text)
    
    ActionChains(browser).move_to_element(row).perform()
    sleep(randint(1,5))

    build_dict = expand_and_read(row)
    relevant_col_dict['שנת בנייה מבנה'].append(build_dict['שנת בנייה מבנה'])
    relevant_col_dict['שנת בנייה דירה'].append(build_dict['שנת בנייה דירה'])
    relevant_col_dict['קומות במבנה'].append(build_dict['קומות במבנה'])

In [ ]:
# nadlan_df = pd.DataFrame.from_dict(relevant_col_dict)

for k,v in relevant_col_dict.items():
    print(f'{k}: {len(v)}')

In [ ]:
# where appartment's year is missing it is likely it is the same as building's year

nadlan_df.loc[nadlan_df['שנת בנייה דירה'] == '-', 'שנת בנייה דירה'] = nadlan_df.loc[nadlan_df['שנת בנייה דירה'] == '-', 'שנת בנייה מבנה']
nadlan_df

In [ ]:
# name your csv
csv_name = 'Neve Shaanan'

nadlan_df.to_csv(f'data/gov/{csv_name}.csv', index=False)